In [ ]:
# Niveen Abdul-Mohsen (bvn9ad)
# Reinforcement Learning (CS 4771) - Figure 6.5 Maximization Bias Q-learning and Double Q-learning 
# Comparison of Q-learning and Double Q-learning on a simple episodic MDP environment
# i used numpy for numerical operations and matplotlib for plotting

import numpy as np
import matplotlib.pyplot as plt






stac 
